In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[4],3b
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,257,257


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  66.806454                    1  44.714664
0.000750        2  30.207494                    2  32.077034
0.001052        3  28.256545                    3  31.059238
0.001476        4  25.342520                    4  28.381462
0.002070        5  22.175199                    5  24.343100
0.002904        6  19.227711                    6  19.785252
0.004074        7  16.723587                    7  15.805634
0.005714        8  14.726586                    8  13.240576
0.008015        9  13.201378                    9  11.987363
0.011243       10  12.057751                   10  11.366014
0.015771       11  11.192190                   11  10.775282
0.022122       12  10.521767                   12  10.027460
0.031031       13  10.093475                   13   9.374695
0.043528       14   9.450069                   14   8.714849
0.061057       15   8.351660                   15   7.990856
0.085645       16   7.344775                   16   7.556328
0.120136       17   6.667402                   17   7.301756
0.168516       18   6.253699                   18   6.959323
0.236378       19   6.090038                   19   6.524059
0.331549       20   6.202514                   20   6.218305
0.465100       21   6.816341                   21   6.511174
0.652400       22   6.516652                   22   6.212923
0.915100       23   5.077246                   23   4.975202
1.283650       24   3.914028                   24   3.888967
1.800600       25   3.128686                   25   3.012596
2.525700       26   2.506895                   26   2.339257
3.542800       27   1.955827                   27   1.949671
4.969550       28   1.572213                   28   1.700042
6.970850       29   1.402256                   29   1.508061
9.778100       30   1.262685                   30   1.299711
13.715850      31   1.074738                   31   1.104909
19.239350      32   0.842086                   32   0.829811
26.987250      33   0.698950                   33   0.668673
37.855300      34   0.648743                   34   0.677328
53.100050      35   0.551780                   35   0.600788
73.887500      36   0.434531                   36   0.431754
97.662500      37   0.339933                   37   0.316804
121.437500     38   0.247797                   38   0.250982
145.212500     39   0.170714                   39   0.198375
168.987500     40   0.126830                   40   0.161923
192.762500     41   0.096019                   41   0.125265
216.537500     42   0.071229                   42   0.089191
240.312500     43   0.047517                   43   0.052791
264.087500     44   0.012414                   44   0.007036
287.862500     45  -0.067561                   45  -0.081450
311.637500     46  -0.102247                   46  -0.120310
335.412500     47  -0.035700                   47  -0.053858
359.187500     48   0.020325                   48   0.005628
382.962500     49   0.037313                   49   0.026595
406.737500     50   0.045267                   50   0.037582
430.512500     51   0.049966                   51   0.044104
454.287500     52   0.052928                   52   0.047094
478.062500     53   0.054903                   53   0.050091
501.837500     54   0.056270                   54   0.051878
525.612500     55   0.057304                   55   0.053632
549.387500     56   0.057981                   56   0.054876
573.162500     57   0.058268                   57   0.055653
596.937500     58   0.057984                   58   0.055927
620.712500     59   0.056861                   59   0.054435
644.487500     60   0.054798                   60   0.053268
668.262500     61   0.051850                   61   0.050094
692.037500     62   0.048270                   62   0.046251
715.812500     63   0.044433                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -14.281746   0.000000 -14.281746 -14.133627 -1.336091e-07   
0.000624    2     -14.283004   0.006200 -14.276805 -14.134868  4.549145e-03   
0.000876    3     -14.283608   0.007703 -14.275905 -14.135404  6.040825e-03   
0.001229    4     -14.284349   0.009625 -14.274724 -14.136073  8.007061e-03   
0.001723    5     -14.285253   0.012014 -14.273239 -14.136898  1.049626e-02   
0.002417    6     -14.286348   0.014933 -14.271416 -14.137909  1.350822e-02   
0.003391    7     -14.287668   0.018469 -14.269198 -14.139136  1.701773e-02   
0.004757    8     -14.289247   0.022754 -14.266493 -14.140618  2.105663e-02   
0.006672    9     -14.291125   0.027973 -14.263152 -14.142390  2.583359e-02   
0.009359    10    -14.293336   0.034387 -14.258950 -14.144489  3.174799e-02   
0.013128    11    -14.295913   0.042347 -14.253567 -14.146942  3.927601e-02   
0.018415    12    -14.298867   0.052309 -14.246557 -14.149758  4.884031e-02   
0.025830    13    -14.302156   0.064841 -14.237315 -14.152890  6.078157e-02   
0.036232    14    -14.305493   0.080615 -14.224877 -14.156067  7.551192e-02   
0.050823    15    -14.308571   0.100028 -14.208544 -14.159020  9.352885e-02   
0.071291    16    -14.311552   0.123257 -14.188295 -14.161886  1.157707e-01   
0.100000    17    -14.314544   0.151227 -14.163317 -14.164741  1.443264e-01   
0.140271    18    -14.317209   0.185698 -14.131511 -14.167228  1.816496e-01   
0.196760    19    -14.318834   0.229170 -14.089664 -14.168603  2.295977e-01   
0.275997    20    -14.317899   0.285397 -14.032502 -14.167345  2.895813e-01   
0.387100    21    -14.310627   0.359756 -13.950871 -14.159724  3.638084e-01   
0.543100    22    -14.283760   0.458850 -13.824910 -14.133345  4.577632e-01   
0.761700    23    -14.229987   0.573825 -13.656163 -14.081211  5.665280e-01   
1.068500    24    -14.161227   0.689585 -13.471642 -14.014555  6.807032e-01   
1.498800    25    -14.083461   0.811325 -13.272136 -13.939114  8.035115e-01   
2.102400    26    -13.995980   0.947548 -13.048433 -13.853635  9.334576e-01   
2.949000    27    -13.899854   1.102827 -12.797027 -13.758961  1.073402e+00   
4.136600    28    -13.805050   1.283168 -12.521882 -13.663842  1.252590e+00   
5.802500    29    -13.721577   1.509952 -12.211625 -13.579256  1.503521e+00   
8.139200    30    -13.641973   1.818492 -11.823482 -13.497454  1.839192e+00   
11.417000   31    -13.561301   2.228093 -11.333208 -13.413405  2.259845e+00   
16.014700   32    -13.494807   2.746933 -10.747873 -13.343031  2.791301e+00   
22.464000   33    -13.495966   3.391417 -10.104549 -13.342984  3.425266e+00   
31.510500   34    -13.592453   4.236914  -9.355538 -13.441232  4.240153e+00   
44.200100   35    -13.736991   5.356626  -8.380365 -13.586996  5.404164e+00   
62.000000   36    -13.907178   6.690255  -7.216923 -13.756194  6.840269e+00   
85.775000   37    -14.061919   8.068772  -5.993147 -13.909040  8.209197e+00   
109.550000  38    -14.134333   9.098546  -5.035787 -13.977292  9.169761e+00   
133.325000  39    -14.162555   9.824642  -4.337912 -13.998806  9.898193e+00   
157.100000  40    -14.195517  10.338390  -3.857127 -14.024047  1.048218e+01   
180.875000  41    -14.237001  10.737068  -3.499933 -14.057107  1.097131e+01   
204.650000  42    -14.289711  11.060198  -3.229514 -14.101408  1.136844e+01   
228.425000  43    -14.358229  11.329320  -3.028910 -14.163346  1.168159e+01   
252.200000  44    -14.452659  11.557572  -2.895087 -14.254891  1.192183e+01   
275.975000  45    -14.614721  11.754595  -2.860126 -14.419203  1.210596e+01   
299.750000  46    -15.004529  11.954130  -3.050399 -14.819464  1.227681e+01   
323.525000  47    -15.675121  12.336760  -3.338361 -15.506153  1.262463e+01   
347.300000  48    -16.454488  13.015586  -3.438902 -16.303810  1.327059e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')